# Chess engine


La verdad que nunca me gusto jugar al ajedrez, ni tampoco me gusta ahora. El objetivo aca es programar algo que funcione, idealmente que me gane (no va a ser dificil).


links
------

wikipedia:

https://en.wikipedia.org/wiki/En_passant

https://en.wikipedia.org/wiki/Fischer_random_chess

https://en.wikipedia.org/wiki/Forsyth%E2%80%93Edwards_Notation

PGN is a sequence of moves from some position that describe a game, while the FEN represents an exact layout of pieces. 

In [10]:
%pip install chess

You should consider upgrading via the '/home/gocandra/workspace/chess/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import chess
import chess.svg
import chess.pgn
import random
from IPython.display import SVG, display

In [12]:
def show_svg(svg):
    display(SVG(svg))

In [16]:
# TODO: fix export pgn (https://www.chess.com/analysis)
def run_game(policy, show_board=False):
    #game = chess.pgn.Game()
    board = chess.Board()        
    while not board.is_game_over():
        next_move = policy(board)
        if not next_move: 
            print("DONE: NO LEGAL MOVES LEFT")
            break
        move = chess.Move.from_uci(next_move)
        board.push(move) 
        if show_board: show_svg(chess.svg.board(board, size=350))
    #game.from_board(board)
    #print(game)
    print(board.outcome())

# Politicas

Una politica es un mapeo de estados a acciones. Una politica busca maximizar o minimizar una cantidad.
Ej: puede maximizar capturas o minimizar cantidad de movimientos en el juego

In [14]:
# esta politica elige un move random
def random_policy(board):
    legal_moves = list(board.generate_legal_moves())
    if len(legal_moves) == 0: return False
    else: 
        return str(random.choice(legal_moves))

run_game(random_policy, False)

Outcome(termination=<Termination.SEVENTYFIVE_MOVES: 4>, winner=None)


In [15]:
# esta politica captura siempre que puede
def greedy_capture_policy(board):
    legal_moves = list(board.generate_legal_moves())
    cap_moves = [move for move in legal_moves if board.is_capture(move)]
    if len(cap_moves) > 0: 
        return str(random.choice(cap_moves))
    elif len(legal_moves) > 0: 
        return str(random.choice(legal_moves))
    else: return False
    
run_game(greedy_capture_policy, False)

Outcome(termination=<Termination.INSUFFICIENT_MATERIAL: 3>, winner=None)


Ninguno de estos parece ser muy bueno, si la greedy_capture_policy captura mas piezas, pero no entiende nada sobre las reglas y objetivos del ajedrez.


que podria hacer?

Puedo montar un arbol completo de todos los posibles tableros, ahre que no, se estiman 7728772977965919677164873487685453137329736522 posiciones posibles (http://talkchess.com/forum3/viewtopic.php?f=7&t=77685)

Osea, tengo que ser capaz de mirar el estado de un tablero y en base a ese estado predecir cual es el movimiento con mas probabilidades de ganar. 
deberia jugar la mayor cantidad de partidas posibles (me puedo bajar un txt de PGN's con partidas ya jugadas?) y que esa experiencia sea queryable. 

me imagino que la politica ve el estado actual del tablero, las acciones posibles y en base a esto decide que accion tomar.

puedo hacer un gpt-3 que juegue complete una secuencia de PGN? 

BASE DE DATOS DE PGN'S:
https://database.lichess.org/

"3,454,824,763 standard rated games, played on lichess.org, in PGN format. Each file contains the games for one month only; they are not cumulative." - messirve

Si puedo filtrarlos por ELO puedo comparar si un modelo entrenado con elos mas altos juega mejor o peor que uno entrenado con bajos o con todos.

# hagamos un modelo baseline con una lstm o algo tranca primero, no tiene sentido ser tan animal y mandarse a armar un gpt-3 por mas placentero que sea



# Implementemos gpt3

Este modelo toma texto como input y genera un token que representa una continuacion plausible del input ej: gpt-3("EL PERRO ES") = "LINDO"

El formato de las partidas PGN es el siguiente:

```
[Event "Rated Bullet tournament https://lichess.org/tournament/yc1WW2Ox"]
[Site "https://lichess.org/PpwPOZMq"]
[Date "2017.04.01"]
[Round "-"]
[White "Abbot"]
[Black "Costello"]
[Result "0-1"]
[UTCDate "2017.04.01"]
[UTCTime "11:32:01"]
[WhiteElo "2100"]
[BlackElo "2000"]
[WhiteRatingDiff "-4"]
[BlackRatingDiff "+1"]
[WhiteTitle "FM"]
[ECO "B30"]
[Opening "Sicilian Defense: Old Sicilian"]
[TimeControl "300+0"]
[Termination "Time forfeit"]

1. e4 { [%eval 0.17] [%clk 0:00:30] } 1... c5 { [%eval 0.19] [%clk 0:00:30] }
2. Nf3 { [%eval 0.25] [%clk 0:00:29] } 2... Nc6 { [%eval 0.33] [%clk 0:00:30] }
3. Bc4 { [%eval -0.13] [%clk 0:00:28] } 3... e6 { [%eval -0.04] [%clk 0:00:30] }
4. c3 { [%eval -0.4] [%clk 0:00:27] } 4... b5? { [%eval 1.18] [%clk 0:00:30] }
5. Bb3?! { [%eval 0.21] [%clk 0:00:26] } 5... c4 { [%eval 0.32] [%clk 0:00:29] }
6. Bc2 { [%eval 0.2] [%clk 0:00:25] } 6... a5 { [%eval 0.6] [%clk 0:00:29] }
7. d4 { [%eval 0.29] [%clk 0:00:23] } 7... cxd3 { [%eval 0.6] [%clk 0:00:27] }
8. Qxd3 { [%eval 0.12] [%clk 0:00:22] } 8... Nf6 { [%eval 0.52] [%clk 0:00:26] }
9. e5 { [%eval 0.39] [%clk 0:00:21] } 9... Nd5 { [%eval 0.45] [%clk 0:00:25] }
10. Bg5?! { [%eval -0.44] [%clk 0:00:18] } 10... Qc7 { [%eval -0.12] [%clk 0:00:23] }
11. Nbd2?? { [%eval -3.15] [%clk 0:00:14] } 11... h6 { [%eval -2.99] [%clk 0:00:23] }
12. Bh4 { [%eval -3.0] [%clk 0:00:11] } 12... Ba6? { [%eval -0.12] [%clk 0:00:23] }
13. b3?? { [%eval -4.14] [%clk 0:00:02] } 13... Nf4? { [%eval -2.73] [%clk 0:00:21] } 0-1
```

Seria loco poder decirle el ELO del jugador y que cometa los mismos errores! 

Me imagino que el primer input al modelo va a ser 


```
X = "
  [Event "Rated Bullet tournament https://lichess.org/tournament/yc1WW2Ox"]
  [Site "https://lichess.org/PpwPOZMq"]
  [Date "2017.04.01"]
  [Round "-"]
  [White "Abbot"]
  [Black "Costello"]
  [Result "0-1"]
  [UTCDate "2017.04.01"]
  [UTCTime "11:32:01"]
  [WhiteElo "2100"]
  [BlackElo "2000"]
  [WhiteRatingDiff "-4"]
  [BlackRatingDiff "+1"]
  [WhiteTitle "FM"]
  [ECO "B30"]
  [Opening "Sicilian Defense: Old Sicilian"]
  [TimeControl "300+0"]
  [Termination "Time forfeit"]

  1.  " 

Y = e4 { [%eval 0.17] [%clk 0:00:30] } 1... c5 { [%eval 0.19] [%clk 0:00:30] }

```

AHora si como funciona realmente un gpt-3?
vamos con nuestro señor y salvador karpathy
https://github.com/karpathy/minGPT

In [ ]:
# BASELINE MINI-MAX ALGORITHM
# https://es.wikipedia.org/wiki/Minimax
